In [1]:
from ollama import chat
from abc import ABC, abstractmethod
import string
import warnings


class Runnables(ABC):
    def __init__(self) -> None:
        pass
    
    @abstractmethod
    def invoke(self):
        pass

class DemoLLM(Runnables):
    def __init__(self, model_name:str ="gemma3:1b"):
        self.model_name = model_name
        self.llm_chat = chat
        
    def invoke(self, prompt:str) :
        self.user_prompt = prompt
        self.llm_response = self.llm_chat(model= self.model_name, 
                                              messages= [
                                                  {"role":"user", 
                                                   'content':self.user_prompt}
                                              ])
        self.llm_response_message = self.llm_response.message
        
        return {'content': self.llm_response_message.content}
        
    def response(self, prompt:str) :
        self.user_prompt = prompt
        self.llm_response = self.llm_chat(model= self.model_name, 
                                              messages= [
                                                  {"role":"user", 
                                                   'content':self.user_prompt}
                                              ])
        self.llm_response_message = self.llm_response.message
        warnings.warn(
        "response() is deprecated and will be removed. Use invoke() instead.",
        category=DeprecationWarning,
        stacklevel=2
        )
        
        return self.llm_response_message

In [2]:
class DemoPromptTemplate(Runnables):
    
    def __init__(self, prompt:str, input_variables: list[str]) -> None:
        self.prompt = prompt
        self.user_input_variables = input_variables
        self.valid_input_variables: list[str] = [var[1] for var in string.Formatter().parse(self.prompt) if isinstance(var[1], str)]
        
        if sorted(self.user_input_variables) != sorted(self.valid_input_variables):
            raise ValueError("prompt variables does not match with input variables")
        
    def invoke(self, variables_dict: dict[str, str]) -> str:
        self.variables_dict = variables_dict
        
        self.formatted_prompt = self.prompt.format(**self.variables_dict)
        return self.formatted_prompt
            
    def format(self, variables_dict: dict[str, str]) -> str:
        self.variables_dict = variables_dict
        
        self.formatted_prompt = self.prompt.format(**self.variables_dict)
        warnings.warn(
        "format() is deprecated and will be removed. Use invoke() instead.",
        category=DeprecationWarning,
        stacklevel=2
        )
        return self.formatted_prompt
            

In [10]:
class DemoStrParser(Runnables):
    def __init__(self) -> None:
        pass

    def invoke(self,llm_message ) -> str:
        self.llm_message = llm_message
        
        return self.llm_message['content']
    
    def parse(self,llm_message ) -> str:
        self.llm_message = llm_message
        warnings.warn(
        "parse() is deprecated and will be removed. Use invoke() instead.",
        category=DeprecationWarning,
        stacklevel=2
        )
        return self.llm_message.content
        

In [11]:
class DemoRunnableConnector(Runnables):
    def __init__(self, runnables: list[any]) -> str:
        self.runnables= runnables
        
    def invoke(self,input_data):
        self.input_data = input_data
        for runnable in self.runnables:
            self.input_data = runnable.invoke(self.input_data)
        return self.input_data

In [12]:
llm = DemoLLM()
parser = DemoStrParser()
prompt_template_1 = DemoPromptTemplate(
    "tell me about {query}",
    ['query']
)

prompt_template_2 = DemoPromptTemplate(
    "Give me questions on {content}",
    ['content']
)

In [19]:
chain_1 = DemoRunnableConnector([prompt_template_1,llm])

chain_2 = DemoRunnableConnector([prompt_template_2,llm, parser])

In [16]:
chain_1.invoke({"query":"Polyethyline"})

{'content': "Okay, let's dive into Polyethylene (PE), also known as polyethylene. It's a hugely important and versatile plastic with a massive range of applications. Here's a breakdown of what you need to know:\n\n**1. What is Polyethylene?**\n\n* **Definition:** Polyethylene is a synthetic polymer – meaning it's made up of repeating molecules – produced by linking together small units (monomers).\n* **Types:** There are many different types of polyethylene, each with slightly different properties. The most common types are:\n    * **Low-Density Polyethylene (LDPE):**  This is the most widely used type, known for its flexibility, good transparency, and ability to be molded into various shapes.\n    * **High-Density Polyethylene (HDPE):**  It’s stronger and more rigid than LDPE, making it ideal for applications requiring durability and impact resistance.\n    * **Linear Low-Density Polyethylene (LLDPE):**  A blend of LDPE and HDPE, offering improved toughness and stretchability.\n\n**2.

In [17]:
chain_2.invoke({"content":"Python"})

{'content': "Okay, let's dive into some Python questions! I'll categorize them by difficulty and topic.  I'll aim for questions that range from beginner to slightly more advanced.\n\n**I. Beginner Level (Focus: Basic Syntax & Data Types)**\n\n1. **What is the difference between a list and a tuple in Python? When would you use one over the other?** (Easy - Tests understanding of fundamental data structures)\n\n2. **What is a variable in Python, and how do you assign a value to it?** (Easy - Core concept)\n\n3. **What are data types in Python? List, Tuple, String, Integer, Float, Boolean, etc.  Give an example of each.** (Easy - Basic data type knowledge)\n\n4. **What is the purpose of the `print()` function in Python?** (Easy - Core functionality)\n\n5. **How do you check if a variable is a number in Python?** (Easy - Basic type checking)\n\n**II. Intermediate Level (Focus: Control Flow & Functions)**\n\n6. **What are 'if' statements in Python?  Provide a simple example of how to use th

In [20]:
connected_chains = DemoRunnableConnector([chain_1, chain_2])

connected_chains.invoke({'query':"Sun"})

'Okay, fantastic! Let’s focus on the **role of the Sun in shaping our weather and daily life.**\n\nHere are some questions for you, based on the provided information, designed to dig deeper into that area:\n\n**1.  Beyond Light and Heat – How does the Sun’s energy actually *influence* weather patterns?** (Think about the mechanisms – think about things like atmospheric pressure, ocean currents, and the jet stream.)\n\n**2.  The Solar Cycle – Can you explain in a way that’s easy to understand *why* the Sun\'s activity (sunspots, flares, etc.) has such a significant impact on Earth’s climate?** (Relate it to the concepts of atmospheric instability and the transfer of energy.)\n\n**3.  What’s the connection between the Sun’s energy and the formation of storms?** (Perhaps you’d like to know how the Sun drives the jet stream and influences the intensity of storms.)\n\n**4.  Imagine a future where we could significantly change the Sun’s activity. What *potential* consequences might arise for